In [6]:
import numpy as np
import os


# Main Parameters
VOXEL_SIZE = 0.05
k_1 = 1
k_2 = 1
threshold = 0.01

In [7]:
from data import load_point_cloud, np_point_cloud2_pcd, pcd2np_point_cloud

# Load Point Clouds and prepocess data
PC1 = "assets/bun000_Cloud.las"
PC2 = "assets/bun045_Cloud.las"
output_path = "output"

point_clouds = [load_point_cloud(PC1), load_point_cloud(PC2)]
accuracies = [0.01, 0.01]

reference_pc = np.vstack(point_clouds)
reference_pcd = np_point_cloud2_pcd(reference_pc)

reference_pcd_ds = reference_pcd.voxel_down_sample(voxel_size=VOXEL_SIZE)
reference_pc_ds = pcd2np_point_cloud(reference_pcd_ds)

reference_pcd_ds_sub = reference_pcd.voxel_down_sample(voxel_size=VOXEL_SIZE / 2)
reference_pc_ds_sub = pcd2np_point_cloud(reference_pcd_ds_sub)

print("Number of points in search space (voxels): ", reference_pc_ds.shape[0])

Number of points in search space (voxels):  25


In [8]:
from weights import compute_weights


weights = compute_weights(
    reference_pcd, point_clouds, accuracies, voxel_size=VOXEL_SIZE
)
print("Computing point cloud weights .... Done")

Computing point cloud weights .... Done


In [9]:
from fusion import weighted_fusion_filter


fused_pc, point_contrib = weighted_fusion_filter(
    point_clouds,
    weights,
    reference_pc_ds,
    reference_pc_ds_sub,
    VOXEL_SIZE,
    k_1,
    k_2,
    threshold,
)

fused_pc = np.unique(fused_pc, axis=0)

100%|██████████| 25/25 [00:02<00:00,  9.34it/s]


In [10]:
from data import write_las_file

file_name = "weighted_fused_filtered_{}_cm_vox.las".format(int(VOXEL_SIZE * 100))
write_las_file(fused_pc, os.path.join(output_path, file_name))